In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt
import sys; sys.path.insert(0, '../')
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
import mne
from esinet import Simulation
from esinet.forward import get_info, create_forward_model
from esinet.util import unpack_fwd
pp = dict(surface='white', hemi='both')

# Get Forward Model

In [2]:
info = get_info(kind='biosemi32')
fwd = create_forward_model(info=info, sampling='ico3')

leadfield, pos = unpack_fwd(fwd)[1:3]
n_chans, n_dipoles = leadfield.shape

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    3.2s remaining:    5.3s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    3.3s remaining:    1.9s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    3.4s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.1s remaining:    0.3s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.4s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.6s remaining:    1.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.6s remaining:    0.3s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.6s finished


# Get sample data

In [24]:
# settings = dict(number_of_sources=1, extents=40, duration_of_trial=0.01, target_snr=99999999999)
settings = dict(number_of_sources=3, extents=(25, 40), duration_of_trial=1, target_snr=2)

sim = Simulation(fwd, info, settings).simulate(2)
stc = sim.source_data[0]
evoked = sim.eeg_data[0].average()
M = evoked.data

brain = stc.plot(**pp)
brain.add_text(0.1, 0.9, 'Ground Truth', 'title',
               font_size=14)

-- number of adjacent vertices : 1284
Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 286.14it/s]


source data shape:  (1284, 1000) (1284, 1000)


100%|██████████| 2/2 [00:00<00:00, 16.81it/s]

Using control points [2.30927956e-09 1.03316335e-08 6.45904458e-08]


For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# Regularization Optimizations

In [25]:
from invert.solvers.minimum_norm_estimates import SolverMinimumNorm

solver = SolverMinimumNorm()

solver.make_inverse_operator(fwd, alpha='auto')
stc_hat = solver.apply_inverse_operator(evoked)
stc_hat.plot(**pp)

solver.make_inverse_operator(fwd, alpha=0)
stc_hat = solver.apply_inverse_operator(evoked)
stc_hat.plot(**pp)

# solver.make_inverse_operator(fwd, alpha=1e10)
# stc_hat = solver.apply_inverse_operator(evoked)
# stc_hat.plot(**pp)

# solver.make_inverse_operator(fwd, alpha=1e11)
# stc_hat = solver.apply_inverse_operator(evoked)
# stc_hat.plot(**pp)





idx = 85, r=27.185882427329428
Using control points [3.95379424e-09 4.75624675e-09 1.60955668e-08]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`
Using control points [6.57975177e-09 7.97935469e-09 2.52496222e-08]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


Using control points [5.28368710e-09 6.21174898e-09 1.14259712e-08]
Using control points [3.41780286e-09 4.01039544e-09 8.91672514e-09]
Using control points [5.28368710e-09 6.21174898e-09 1.14259712e-08]
Using control points [1.46377819e-09 5.15575336e-09 1.84379291e-08]


In [34]:
from mne.minimum_norm import make_inverse_operator as mne_inverse
from mne.minimum_norm import apply_inverse as mne_apply
from mne import make_ad_hoc_cov
noise_cov = make_ad_hoc_cov(evoked.info)
mne_io = mne_inverse(evoked.info, fwd, noise_cov=noise_cov, fixed=True, loose=0, depth=0)
stc_hat = mne_apply(evoked, mne_io, method="MNE")
stc_hat.plot(**pp)

Computing inverse operator with 32 channels.
    32 out of 32 channels remain after picking
Selected 32 channels
Whitening the forward solution.
    Created an SSP operator (subspace dimension = 1)
Computing rank from covariance with rank=None
    Using tolerance 2.8e-18 (2.2e-16 eps * 32 dim * 0.0004  max singular value)
    Estimated rank (eeg): 31
    EEG: rank 31 computed from 32 data channels with 1 projector
    Setting small EEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.
Computing SVD of whitened and weighted lead field matrix.
    largest singular value = 3.25404
    scaling factor to adjust the trace = 3.07424e+21 (nchan = 32 nzero = 1)
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 31 (1 small eigen

Using control points [1.39855642e-09 1.65036569e-09 4.33651143e-09]
Using control points [3.41780286e-09 4.01039544e-09 8.91672514e-09]
Using control points [5.28368710e-09 6.21174898e-09 1.14259712e-08]
Using control points [1.39855642e-09 1.65036569e-09 4.33651143e-09]
Using control points [3.63611051e-09 4.27780042e-09 1.07905916e-08]
Using control points [1.99579652e-09 2.35524405e-09 6.61545605e-09]
